In [1]:
import os
import numpy as np
from PIL import Image
import json
import tensorflow as tf
import cv2
import random
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
tf.__version__

'2.6.0'

In [3]:
#전처리 된 파일리스트 변수 선언 코드

dir_path_pre_TS_damage_part = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage_part'
file_list_pre_TS_damage_part = os.listdir(dir_path_pre_TS_damage_part)
print("file_list_pre_TS_damage_part :",len(file_list_pre_TS_damage_part))

dir_path_pre_TS_damage = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage'
file_list_pre_TS_damage = os.listdir(dir_path_pre_TS_damage)
print("file_list_pre_TS_damage :",len(file_list_pre_TS_damage))

dir_path_pre_TL_damage_part = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage_part'
file_list_pre_TL_damage_part = os.listdir(dir_path_pre_TL_damage_part)
print("file_list_pre_TL_damage_part :",len(file_list_pre_TL_damage_part))

dir_path_pre_TL_damage = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage'
file_list_pre_TL_damage = os.listdir(dir_path_pre_TL_damage)
print("file_list_pre_TL_damage :",len(file_list_pre_TL_damage))

dir_path_pre_견적서 = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/견적서'
file_list_pre_견적서 = os.listdir(dir_path_pre_견적서)
print("견적서 파일 개수 :",len(file_list_pre_견적서))

file_list_pre_TS_damage_part : 32993
file_list_pre_TS_damage : 50759
file_list_pre_TL_damage_part : 32994
file_list_pre_TL_damage : 50760
견적서 파일 개수 : 24012


In [4]:
dir_path_pre_TS_damage_part_demo = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage_part_demo'
file_list_pre_TS_damage_part_demo = os.listdir(dir_path_pre_TS_damage_part_demo)
print("file_list_pre_TS_damage_part_demo :",len(file_list_pre_TS_damage_part_demo))

dir_path_pre_TL_damage_part_demo = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage_part_demo'
file_list_pre_TL_damage_part_demo = os.listdir(dir_path_pre_TL_damage_part_demo)
print("file_list_pre_TS_damage_part_demo :",len(file_list_pre_TL_damage_part_demo))

file_list_pre_TS_damage_part_demo : 8000
file_list_pre_TS_damage_part_demo : 8000


In [5]:
dir_path_pre_TS_damage_part_val = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage_part_val'
file_list_pre_TS_damage_part_val = os.listdir(dir_path_pre_TS_damage_part_val)

dir_path_pre_TL_damage_part_val = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage_part_val'
file_list_pre_TL_damage_part_val = os.listdir(dir_path_pre_TL_damage_part_val)

print("file_list_pre_TS_damage_part_val :",len(file_list_pre_TS_damage_part_val))
print("file_list_pre_TL_damage_part_val :",len(file_list_pre_TL_damage_part_val))
# # 결과 출력하기
# print(random_files_TS_damage_part)

file_list_pre_TS_damage_part_val : 2000
file_list_pre_TL_damage_part_val : 2000


In [6]:
BATCH_SIZE = 16

In [7]:
def get_data_training(path):
                                                         
    def make_image_dataset(img_name):
        image_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage_part_demo/' + img_name +'.jpg'
        image = Image.open(image_path)
        image = image.resize((256, 256)) #input에 맞게 나중에 수정할 것
        image = np.array(image)
        image = image / 255.0  # 정규화 추가
        return image
    
    
    
    def make_segmentation_map_dataset(json_file):
        json_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage_part_demo/' + json_file + '.json'
        with open(json_path, 'r', encoding='UTF-8') as json_file:
            json_data = json.load(json_file)

        img = np.zeros((json_data['images']['height'], json_data['images']['width']), np.uint8)

        for ann in json_data['annotations']:
            if 'segmentation' in ann:
                segmentation = ann['segmentation'][0][0]
                cv2.fillPoly(img, [np.array(segmentation)], (255, 255, 255)) #흰색
                
        img = cv2.resize(img, (256,256,)) #input에 맞게 나중에 수정할 것
        img = img / 255.0
        img = np.expand_dims(img, axis=-1)  # 마지막 차원 추가
        return img
    
    
    def generator():
        for filename in os.listdir(path):
            final_img = make_image_dataset(filename.split('.')[0])
            final_map = make_segmentation_map_dataset(filename.split('.')[0])
            yield (final_img, final_map)
    
        
    
    dataset =  tf.data.Dataset.from_generator(generator,
                                              (tf.float64, tf.float64))
    
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
#     result = dataset.__iter__()
#     return result
    return dataset
# tf.data.Dataset 객체를 반환

In [8]:
datasets_TS_damage_part_demo = get_data_training(dir_path_pre_TS_damage_part_demo)

In [9]:
# import matplotlib.pyplot as plt

# fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))
# for i, ax in enumerate(axes.flat):
    
#     ax.imshow(final_map_demo[i].numpy().astype(int))
#     ax.axis('off')

# plt.show()

In [10]:
def get_data_val(path):
                                                         
    def make_image_dataset(img_name):
        image_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/원천데이터/TS_damage_part_val/' + img_name +'.jpg'
        image = Image.open(image_path)
        image = image.resize((256, 256)) #input에 맞게 나중에 수정할 것
        image = np.array(image)
        image = image / 255.0  # 정규화 추가
        return image
    
    def make_segmentation_map_dataset(json_file):
        json_path = os.getenv('HOME')+'/aiffel/blackbox/preprocessed_training/라벨링데이터/TL_damage_part_val/' + json_file + '.json'
        with open(json_path, 'r', encoding='UTF-8') as json_file:
            json_data = json.load(json_file)

        img = np.zeros((json_data['images']['height'], json_data['images']['width'], ), np.uint8)

        for ann in json_data['annotations']:
            if 'segmentation' in ann:
                segmentation = ann['segmentation'][0][0]
                cv2.fillPoly(img, [np.array(segmentation)], (255, 255, 255)) #흰색
        img = cv2.resize(img, (256,256,)) #input에 맞게 나중에 수정할 것
        img = img / 255.0
        img = np.expand_dims(img, axis=-1)  # 마지막 차원 추가
        return img
    
    def generator():
        for filename in os.listdir(path):
            final_img = make_image_dataset(filename.split('.')[0])
            final_map = make_segmentation_map_dataset(filename.split('.')[0])
            yield (final_img, final_map)
    
        
    
    dataset =  tf.data.Dataset.from_generator(generator,
                                              (tf.float64, tf.float64))
    
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
#     result = dataset.__iter__()
#     return result
    return dataset
# tf.data.Dataset 객체를 반환

In [11]:
datasets_TS_damage_part_val = get_data_val(dir_path_pre_TS_damage_part_val)

In [12]:
train_dataset = datasets_TS_damage_part_demo
test_dataset = datasets_TS_damage_part_val

In [13]:
def double_conv_block(x, n_filters):

   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
   # Conv2D then ReLU activation
   x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)

   return x

In [14]:
def downsample_block(x, n_filters):
   f = double_conv_block(x, n_filters)
   p = layers.MaxPool2D(2)(f)
   p = layers.Dropout(0.3)(p)

   return f, p

In [15]:
def upsample_block(x, conv_features, n_filters):
   # upsample
   x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
   # concatenate
   x = layers.concatenate([x, conv_features])
   # dropout
   x = layers.Dropout(0.3)(x)
   # Conv2D twice with ReLU activation
   x = double_conv_block(x, n_filters)

   return x

In [16]:
def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(256,256,3))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)

    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # outputs
    outputs = layers.Conv2D(1, 1, padding="same", activation = "sigmoid")(u9)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model

In [17]:
unet_model = build_unet_model()

In [18]:
from tensorflow.keras import optimizers
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf

METRICS = [tf.keras.metrics.BinaryAccuracy()]

unet_model.compile(optimizer='adam',
              loss=BinaryCrossentropy(),
              metrics=METRICS)


https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy

## 모델 학습

In [ ]:
NUM_EPOCHS = 6

# 체크포인트 저장 경로 및 파일 이름 지정
checkpoint_path = os.path.join(os.getenv('HOME'), 'aiffel/blackbox/checkpoints', 'cp-{epoch:04d}.ckpt')
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 정의
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    save_weights_only=True, 
    save_freq=2 # 2 에포크마다 체크포인트 저장
)

history = unet_model.fit(
    train_dataset, 
    epochs=NUM_EPOCHS,
    validation_data=test_dataset,
    callbacks=[cp_callback],
    verbose=1
)

Epoch 1/6
     58/Unknown - 178s 3s/step - loss: 0.5399 - binary_accuracy: 0.7896

### 모델학습 (이전코드)

In [ ]:
history.history

In [ ]:
# for images, masks in test_dataset.take(2):
#     sample_image, sample_mask = images[1], masks[1]
#     sample_mask = tf.reshape(sample_mask, shape=(256, 256, 1))
#     display([sample_image, sample_mask])

In [ ]:
# def display(display_list):
#   plt.figure(figsize=(15, 15))

#   title = ['Input Image', 'True Mask', 'Predicted Mask']

#   for i in range(len(display_list)):
#     plt.subplot(1, len(display_list), i+1)
#     plt.title(title[i])
#     plt.imshow(tf.keras.utils.array_to_img(display_list[i], data_format='channels_last'))
#     plt.axis('off')
#   plt.show()

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for final_img_demo, final_map_demo in dataset.take(num):
            pred_mask = model.predict(image)
            display([final_img[0], final_map[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])
        print(model.predict(sample_image[tf.newaxis, ...]))

In [ ]:
show_predictions()

아하 마스크가 안나오면 다시 255를 곱하면 될수도.

In [ ]:
def display(display_list):
  # 이미지 출력에 대한 설정
  plt.figure(figsize=(15, 15))
  title = ['Input Image', 'True Mask', 'Predicted Mask']
  # display_list의 각 이미지를 출력
  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i], data_format='channels_last'))
    plt.axis('off')
  plt.show()

def create_mask(pred_mask):
  # 예측 결과 중 가장 높은 값의 인덱스로부터 마스크 생성
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=1):
    if dataset:
        # 데이터셋으로부터 이미지와 마스크를 가져와서 예측 결과 시각화
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        # 샘플 이미지와 마스크를 이용하여 예측 결과 시각화
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])


In [ ]:
# 데이터셋으로부터 예측 결과 시각화
show_predictions(dataset=datasets_TS_damage_part_demo, num=1)

# 샘플 이미지로부터 예측 결과 시각화
show_predictions()